# IoT Stream Zeiss 2022

This notebook is about storing and accessing data to/from a SQL database that is hosted on Azure. You will be guided through some examples. Afterwards you apply the learned to your use-case.

Explanation of terms:

**Azure:** Azure is the cloud service from Microsoft. A similar cloud service is AWS from Amazon.
<br>
**Azure Database:** We use a relational Database on Azure (PostgreSQL - Single Server) to store the collected data. A relational database is a collection of data items with pre-defined relationships between them. These items are organized as a set of tables with columns and rows. Tables are used to hold information about the objects to be represented in the database.
<br>
**SQL:** SQL is a standard language for storing, manipulating and retrieving data in databases. We will use it to communicate with our database.
<br>The SQL commands can be executed with `cursor.execute( SQLcommand )`

**For SQL basics see Notebook 00_Basics.ipynb**

**Steps overview:**
<br>&emsp; `Step 1: Setup`
<br>&emsp;&emsp; - import libraries
<br>&emsp;&emsp; - create a connection to your database
<br>&emsp; `Step 2: Store data`
<br>&emsp;&emsp; - create a table in your database
<br>&emsp;&emsp; - collect and store data into your database
<br>&emsp; `Step 3: Access and visualize data`
<br>&emsp;&emsp; - access the data in the database
<br>&emsp;&emsp; - create plots of the data
<br>&emsp; `Step 4: Close the connection to the database`
<br>&emsp;&emsp; - after all the work you just have to close the connection to your database
<br>&emsp; `Step 5: Apply - Create the technical setup for your use-case`
<br>&emsp;&emsp; - set up a table according to your use-case requirements
<br>&emsp;&emsp; - collect data for your use-case and store it in the database

# Step 1: Setup

Import the needed libraries:

In [ ]:
import psycopg2
import pandas as pd
import time
import random
from datetime import datetime
from datetime import timezone
from time import strftime, gmtime

The connect() function creates a new database session and returns a new instance of connection. To build a connection, you need the following data:

- Username
- Password
- Host Name: This is the server name or Ip address on which databse is running
- Database Name: Database name to which you want to connect - in our case: 'iot_db_XX' where XX is your group number
- Port

The cursor allows interaction with the database:

- send commands to the database using execute() and executemany() methods.
- retrieve data from the database using methods such as fetchone(), fetchmany(), fetchall(), or by iteration.

Please **fill in the password and Database Name** for the database and execute the following cell:

The Database Name is 'iot_db_XX' where XX is your group number (e.g. Group 1: iot_db_01)

In [ ]:
# create connection
conn = psycopg2.connect(dbname='', # FILL IN THE DATABASE NAME OF YOUR GROUP
                        user='nimda@iot-zeiss-2504', 
                        host='iot-zeiss-2504.postgres.database.azure.com', 
                        password='', # FILL IN THE DATABASE PASSWORD
                        port='5432', 
                        sslmode='require')

# create cursor
cursor = conn.cursor()
print("Connection established")

If you receive the message "current transaction is aborted, commands ignored until end of transaction block", then you have to start a new connection/cursor.

# Step 2: Store data

### Drop previous table of same name if one exists

In case you want to start over, you can delete your table

**Important:**

WITH THIS COMMAND YOU PERMANENTLY DELETE THE TABLE, MAKE SURE YOU DON'T ACCIDENTALLY DELETE IT!

In [ ]:
# ONLY RUN THIS CELL IN CASE YOU WANT TO DELETE YOUR TABLE

cursor.execute("DROP TABLE IF EXISTS test_table;")

print("Finished dropping table (if existed)")

### Create a table

**Please note:**

In the tutorial you will create a table named `test_group_0`. Please replace the zero by the number of your group.

When creating a table, you need to define the table name after `CREATE TABLE`. In the brackets the columns of the table are defined. The column name is followed by the data type of the column. 

In [ ]:
cursor.execute("""
CREATE TABLE test_table(
    name_column1 FLOAT8, 
    name_column2 FLOAT8, 
    name_column3 FLOAT8, 
    date DATE, 
    time TIME
    );
""")
           
print("Finished creating table")

### Collect and insert data into the table

The command cursor.execute() enables you to execute an SQL query, such as `INSERT INTO`. The function takes as an input the query itself and the data you want to insert. The values in the INSERT INTO query are replaced by placeholder `%s` (e.g. `VALUES (%s, %s)`), the variables containing the data are placed in separate bracets (e.g. `(sensor_value_1, sensor_value_2)`). The numbers of `%s` and input variables need to be identical. Don't forget the comma between the SQL query and the data variables.

Read and try to understand the following code. Then execute the cell and after some time stop the cell with the interrupt button (next to play button)

In [ ]:
# start infinite loop
while True:
    try:
        # randomly assign numbers between 0 and 100 to the sensor values - this needs to be replaced by your sensor values in your project
        sensor_value_1 = random.randint(0,100)
        sensor_value_2 = random.randint(0,100)
        sensor_value_3 = random.randint(0,100)

        # assign date and time to track when the data was collected
        date_value = datetime.today().strftime("%Y-%m-%d")
        time_value = datetime.today().strftime("%H:%M:%S")

        # insert data into the database (don't forget to replace "test_group_0" with your group number)
        # when data is inserted, no data type is needed
        cursor.execute("""
        INSERT INTO test_table (
            name_column1, 
            name_column2, 
            name_column3,
            date, 
            time) 
        VALUES (%s, %s, %s, %s, %s);""", (sensor_value_1, sensor_value_2, sensor_value_3, date_value, time_value))

        # HINT: the number of columns, the number of "%s" and the number of values entered needs to match

        # print the values that the sensor collected
        print(f"Sensor 1 = {sensor_value_1} \t Sensor 2 = {sensor_value_2} \t Sensor 3 = {sensor_value_3} \t Date = {date_value} \t Time = {time_value}")

        # wait for 1 second and increase counter i by 1
        time.sleep(1)
    except KeyboardInterrupt:
        break

# Step 3: Access and visualize data

### Read data from database

In [ ]:
# Fetch all rows from table (test_group_0 needs to be replaced by your group number)
cursor.execute("""
    SELECT 
        name_column1, 
        name_column2, 
        name_column3,
        date, 
        time 
    FROM test_table;
    """)

rows = cursor.fetchall()
    
# Create a dataframe with rows of the database table
df = pd.DataFrame(rows, columns=["name_column1", "name_column2", "name_column3", "date", "time"])

# show the first 10 rows of the dataframe
df.head(10)

### Plot data 

In [ ]:
# create a histogram of the column 'air_sensor' of the dataframe
df["name_column1"].hist()

# Step 4: Close the connection to the database

Don't forget to close your connection to the database, once you are done. 

**Hint:**
It might be a good idea to include the openning/closing of the connection at the beginning/end of each code snippit. Like this you can avoid connection issues.

In [ ]:
conn.commit()
cursor.close()
conn.close()
print("connection closed")

# Code example with weather station

In [ ]:
# open connection to database
conn = psycopg2.connect(
    dbname='', # FILL IN THE DATABASE NAME OF YOUR GROUP
    user='nimda@iot-zeiss-2504', 
    host='iot-zeiss-2504.postgres.database.azure.com', 
    password='', # FILL IN THE DATABASE PASSWORD
    port='5432', 
    sslmode='require')
cursor = conn.cursor()
print("Connection established")

# Create the table weather_station:
cursor.execute("""CREATE TABLE weather_station(
temperature_column INT, 
humidity_column INT, 
date DATE, 
time TIME);
""")           
print("Finished creating table")

# Close the connection
conn.commit()
cursor.close()
conn.close()
print("connection closed")

In [ ]:
import time
import grovepi
import grove_rgb_lcd
import math

# Sensor connections:
temp_humidity_sensor = 4  # The Sensor goes on digital port D4.

# Define Pin Mode
grovepi.pinMode(temp_humidity_sensor,"INPUT")

# open connection to database
conn = psycopg2.connect(
    dbname='', # FILL IN THE DATABASE NAME OF YOUR GROUP
    user='nimda@iot-zeiss-2504', 
    host='iot-zeiss-2504.postgres.database.azure.com', 
    password='', # FILL IN THE DATABASE PASSWORD
    port='5432', 
    sslmode='require')
cursor = conn.cursor()
print("Connection established")

# loop to collect data
while True:
    try:      
        # Get sensor value 
        [temp_value,humidity_value] = grovepi.dht(temp_humidity_sensor,0)

        # set date and time
        date_value = datetime.today().strftime("%Y-%m-%d")
        time_value = datetime.today().strftime("%H:%M:%S")
        
        # print the values that the sensor collected
        print(f"Temperatur = {temp_value}C \t Humidity = {humidity_value}%")
        
        # insert data into your project database
        cursor.execute("""
        INSERT INTO weather_station (
            temperature_column, 
            humidity_column, 
            date, 
            time) 
        VALUES (%s, %s, %s, %s);""", (temp_value, humidity_value, date_value, time_value))
        
        # wait 1 second        
        time.sleep(1)       

    except IOError:
        print ("Error")

    except KeyboardInterrupt:
        # close connection
        conn.commit()
        cursor.close()
        conn.close()
        print("connection closed")
        break

# Step 5: Apply - Create the technical setup for your use-case

You have completed the tutorial and you are now ready to go up and adjust the code according to your use-case. Good Luck :-)